In [4]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM


tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-ru-en")

model = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-ru-en")



###### Загрузка сохраненной модели #########

# saved_model_dir = "./trained_model"

# # Загрузка модели и токенизатора
# tokenizer = AutoTokenizer.from_pretrained(saved_model_dir)
# model = AutoModelForSeq2SeqLM.from_pretrained(saved_model_dir)

In [6]:
import torch
use_cuda = torch.cuda.is_available()

device = torch.device("cuda:0" if use_cuda else "cpu")
model = model.to(device=device)
print(use_cuda)

AssertionError: Torch not compiled with CUDA enabled

In [3]:
path = "words_correction"
# Пути к файлам с данными
train_data_file = path + "/train_data.txt"
train_data1_file = path + "/train_data_1.txt"
val_data_file = path + "/val_data.txt"

# Функция для чтения данных из файла
def read_data(file_path):
    with open(file_path, "r", encoding="utf-8") as f:
        data = f.readlines()
    return data

# Чтение данных для обучения и валидации
train_data = read_data(train_data_file) + read_data(train_data1_file)
val_data = read_data(val_data_file)

# Преобразование данных в формат, понимаемый Seq2SeqDataset
def process_data(data):
    processed_data = []
    for line in data:
        source_text, target_text = line.strip().split(":")
        processed_data.append({"source_text": source_text.strip(), "target_text": target_text.strip()})
    return processed_data

train_processed_data = process_data(train_data)
val_processed_data = process_data(val_data)

from torch.utils.data import Dataset

class CustomDataset(Dataset):
    def __init__(self, data, tokenizer, max_source_length, max_target_length):
        self.data = data
        self.tokenizer = tokenizer
        self.max_source_length = max_source_length
        self.max_target_length = max_target_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        source_text = self.data[index]["source_text"]
        target_text = self.data[index]["target_text"]

        # Токенизация и добавление специальных токенов
        source_inputs = self.tokenizer.encode_plus(source_text, max_length=self.max_source_length, padding="max_length", truncation=True, return_tensors="pt")
        target_inputs = self.tokenizer.encode_plus(target_text, max_length=self.max_target_length, padding="max_length", truncation=True, return_tensors="pt")

        source_ids = source_inputs["input_ids"].squeeze()
        source_mask = source_inputs["attention_mask"].squeeze()
        target_ids = target_inputs["input_ids"].squeeze()

        return {
            "input_ids": source_ids,
            "attention_mask": source_mask,
            "labels": target_ids,
        }

# Создание датасетов
train_dataset = CustomDataset(train_processed_data, tokenizer, max_source_length=128, max_target_length=128)
val_dataset = CustomDataset(val_processed_data, tokenizer, max_source_length=128, max_target_length=128)




FileNotFoundError: [Errno 2] No such file or directory: 'words_correction/train_data.txt'

In [ ]:
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir='./results',
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    predict_with_generate=True,
    logging_dir='./logs',
    logging_steps=1000,
    evaluation_strategy="steps",
    save_steps=1000,
    num_train_epochs=20,
    overwrite_output_dir=True,
    # Добавьте любые другие аргументы, необходимые для fine-tuning
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,  # ваш тренировочный датасет
    eval_dataset=val_dataset,     # ваш валидационный датасет
    tokenizer=tokenizer
)

trainer.train()

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


Step,Training Loss,Validation Loss
1000,0.058900,0.048341
2000,0.006200,0.050806


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 512, 'num_beams': 6, 'bad_words_ids': [[62517]], 'forced_eos_token_id': 0}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 512, 'num_beams': 6, 'bad_words_ids': [[62517]], 'forced_eos_token_id': 0}


Step,Training Loss,Validation Loss
1000,0.058900,0.048341
2000,0.006200,0.050806
3000,0.002700,0.054486


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 512, 'num_beams': 6, 'bad_words_ids': [[62517]], 'forced_eos_token_id': 0}


TrainOutput(global_step=3220, training_loss=0.021192953397768624, metrics={'train_runtime': 394.2568, 'train_samples_per_second': 32.618, 'train_steps_per_second': 8.167, 'total_flos': 435932972974080.0, 'train_loss': 0.021192953397768624, 'epoch': 20.0})

## Тестирование

In [ ]:

def interact_with_model(model, tokenizer, input_text):
    # Предобработка входных данных
    input_ids = tokenizer.encode(input_text, return_tensors="pt").to(device=device)

    # Передача данных в модель
    output_ids = model.generate(input_ids, max_length=128, num_beams=4, early_stopping=True)

    # Декодирование выходных данных
    output_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    for i in output_ids:
      print(tokenizer.decode(i))

    return output_text


In [ ]:
input_text = ""
output_text = interact_with_model(model, tokenizer, input_text)
print("Восстановленное сокращение:", output_text)

<pad>  автор</s>
Восстановленное сокращение: автор


In [ ]:
# Путь для сохранения модели
output_model_dir = "./trained_model"

# Сохранение модели и токенизатора
model.save_pretrained(output_model_dir)
tokenizer.save_pretrained(output_model_dir)
